In [11]:
# Step 1: Install Required Libraries
# !pip install faiss-cpu langchain langchain-community 

In [1]:
# Required Imports
import boto3

# Bedrock
from langchain_community.embeddings import BedrockEmbeddings

# Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Pdf Loader
from langchain_community.document_loaders import PyPDFLoader

# Import FAISS
from langchain_community.vectorstores import FAISS

In [2]:
# Creating Bedrock Embdedding Client
bedrock_client = boto3.client(service_name="bedrock-runtime",  region_name="us-east-1")
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

In [3]:
# Loading Pdf
loader = PyPDFLoader("sample.pdf")
pages = loader.load_and_split()
# print(pages)

In [4]:
# Split the pages / text into chunks
def split_text(pages, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(pages)
    return docs

In [5]:
splitted_docs = split_text(pages, 1000, 200)

In [6]:
print(len(splitted_docs))
# print(splitted_docs[0])

13


In [7]:
# create vector store and save index locally
def create_vector_store(request_id, documents):
    index=FAISS.from_documents(documents, bedrock_embeddings)
    file_name=f"{request_id}"
    folder_path="./"
    index.save_local(index_name=file_name, folder_path=folder_path)

In [8]:
# creating vectorstore kb
create_vector_store("faiss_vdb", splitted_docs)